In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re

In [3]:
from sqlalchemy import create_engine

username = 'admin'
password = 'nX0d4tuvxL3w5hMjJRfz'
host = 'ai-engine.cluster-cla9jbugaywl.eu-west-2.rds.amazonaws.com'
database_name = 'superleague_clubs'

connection_url = f'mysql+pymysql://{username}:{password}@{host}/{database_name}'

def insert_into_db(df, table_name) : 

    engine = create_engine(connection_url)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False) 
    engine.dispose()
    return 'Data inserted successfully'

In [4]:
def split_at_first_capital(input_string):
    match = re.search(r'[A-Z]', input_string)
    if match:
        index = match.start()
        return input_string[index:]
    else:
        return input_string, ''

In [5]:
def extract_country_name(input_string):
    match = re.match(r'([a-z]{2,3})([A-Za-z.\s]+)', input_string)
    
    if match:
        country_name = match.group(2)
        return country_name.strip()
    else:
        return input_string

In [6]:
def get_club_data(club_url, season, club):

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu") 

    driver = webdriver.Chrome(options=chrome_options)  
    driver.get(club_url)
    driver.implicitly_wait(5)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    driver.quit()

    try : 
        players_container = soup.find("div", {"id": "all_stats_standard"})
        player_table = players_container.find("table")

        header_row = player_table.find("thead").find_all("th")
        headers = [cell.get_text(strip=True) for cell in header_row]
        to_remove = ['', '', 'Performance','Playing Time', 'Expected', 'Progression','Per 90 Minutes', '', ' ']
        for val in to_remove:
            if val in headers: 
                headers.remove(val)

        data_rows = player_table.find("tbody").find_all("tr")

        player_data = []

        for row in data_rows:
            row_data = []
            columns = row.find_all(["td", "th"])  # Include both <td> and <th> elements

            for header, column in zip(headers, columns):
                if header == 'Player':
                    row_data.append(column.find('td' > 'a')['href']) if column.find('td' > 'a') else ''
                    row_data.append(column.find('td' > 'a').get_text(strip=True)) if column.find('td' > 'a') else ''
                    row_data.append(season)
                    row_data.append(club)
                if header == 'Nation':
                    row_data.append(column.find('td' > 'a').get_text(strip=True)) if column.find('td' > 'a') else ''
                if header == 'Pos':
                    row_data.append(column.get_text(strip=True)) if column.get_text() else ''
                if header == 'Matches':
                    row_data.append(column.find('td' > 'a')['href']) if column.find('td' > 'a') else ''

            player_data.append(row_data)

    except Exception as e: 
        print(f'Player data not available for season : {season}')
        return None
        
    try : 
        matches_container = soup.find("div", {"id": "div_matchlogs_for"})
        matches_table = matches_container.find("table")

        header_row = matches_table.find("thead").find("tr")
        header_cells = header_row.find_all("th")
        headers = [cell.get_text(strip=True) for cell in header_cells]

        data_rows = matches_table.find("tbody").find_all("tr")
        match_data = []

        for row in data_rows:
            row_data = []
            columns = row.find_all(["td", "th"])  # Include both <td> and <th> elements
            
            for header, column in zip(headers, columns):

                if header == 'Home':
                    home = column.find('td' > 'home_team')
                    row_data.append(home.text if home else '')

                elif header == 'Away':
                    away = column.find('a')
                    row_data.append(away.text if away else '')

                elif header == 'Match Report':
                    match = column.find('a')
                    row_data.append(match['href'] if match else '')

                else: 
                    row_data.append(column.get_text(strip=True))

            match_data.append(row_data)

        matches_df = pd.DataFrame(match_data, columns=headers)
        matches_df = matches_df[matches_df['Match Report'] != 'Match Report']
        matches_df = matches_df[matches_df['Match Report'] != '']
        matches_df = matches_df.dropna()
        #matches_df.drop(columns= 'xG', inplace= True)
        matches_df["fbref_matchid"] = matches_df['Match Report'].str.extract(r'/matches/(\w{8})/')
        matches_df["Time"] = matches_df["Time"].str.split('(', n=1).str[0]
        matches_df["season"] = season
        matches_df["club"] = club

    except Exception as e: 
        print(f'Club matches not available for season : {season}')
        return None

    return player_data, matches_df


In [7]:
def get_player_data(url, season) : 

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu") 

    driver = webdriver.Chrome(options=chrome_options)   
    driver.get(url)
    driver.implicitly_wait(5)

    page_source = driver.page_source
    driver.quit()

    soup = BeautifulSoup(page_source, "html.parser")

    try : 
        table_container = soup.find("div", {"id": "all_matchlogs_all"})
        table = table_container.find("table")

        data = [] 
        headers = []

        header_row = table.find("thead").find_all("tr")[1]
        header_cells = header_row.find_all("th")
        headers = [cell.get_text(strip=True) for cell in header_cells]

        data_rows = table.find("tbody").find_all("tr")

        for rowval in data_rows:
            row_data = []
            columns = rowval.find_all(["td", "th"]) 
            
            for column in columns:
                link_element = column.find("a")
                cell_text = column.get_text(strip=True)
                if link_element and cell_text == 'Match Report':
                    row_data.append(link_element["href"])
                else:
                    row_data.append(column.get_text(strip=True))

            data.append(row_data)

        pm_df = pd.DataFrame(data, columns=headers)
        pm_df = pm_df[['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'CrdY', 'CrdR', 'Match Report']]
        pm_df["fbref_matchid"] = pm_df['Match Report'].str.extract(r'/matches/(\w{8})/')

        # pm_df['Squad'] = pm_df['Squad'].apply(extract_country_name)
        # pm_df['Opponent'] = pm_df['Opponent'].apply(extract_country_name)

        pm_df['Squad'] = pm_df['Squad'].astype(str).apply(extract_country_name)
        pm_df['Opponent'] = pm_df['Opponent'].astype(str).apply(extract_country_name)
        pm_df['Opponent'] = pm_df['Opponent'].apply(split_at_first_capital)
        pm_df['Season'] = season

        return pm_df

    except Exception as e : 

        print(f'player data not available for : {url}')
        return None


In [8]:
club = 'Internazionale'
club_fbref_id = 'd609edc0'
players_list = []
matches_frame = []

season_start = 1989
season_end = 2022

for year in range(season_end, season_start, -1):
    season = f'{year}-{year+1}'
    url = f'https://fbref.com/en/squads/{club_fbref_id}/{season}/{club}-Stats'
    print(url)
    try : 
        lst, dfm = get_club_data(url, season, club)
        players_list.append(lst)
        matches_frame.append(dfm)
    except Exception as e : 
        print(f"Could not fetch data for {url}")

https://fbref.com/en/squads/d609edc0/2022-2023/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2021-2022/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2020-2021/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2019-2020/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2018-2019/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2017-2018/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2016-2017/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2015-2016/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2014-2015/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2013-2014/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2012-2013/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2011-2012/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2010-2011/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2009-2010/Internazionale-Stats
https://fbref.com/en/squads/d609edc0/2008-2009/I

In [9]:
matches_final = []

for df in matches_frame :
    df1 = df[['Comp', 'Round', 'Day', 'Date', 'Time', 'Result', 'GF', 'GA', 'Opponent', 'Venue', 'Captain', 'Referee', 'Match Report', 'fbref_matchid', 'season', 'club']]
    matches_final.append(df1)

match_reports = pd.concat(matches_final, ignore_index=True)
match_reports['Opponent'] = match_reports['Opponent'].astype(str).apply(extract_country_name)
match_reports['Opponent'] = match_reports['Opponent'].apply(split_at_first_capital)

In [9]:
flat_data = [item for sublist in players_list for item in sublist]
players_final = pd.DataFrame(flat_data, columns = ['PlayerURL', 'Player', 'Season', 'Club', 'Nation', 'Position', 'MatchLogs'])
players_final["fbref_playerid"] = players_final['PlayerURL'].str.extract(r'/players/(\w{8})/')

In [10]:
insert_into_db(match_reports, f'${club}-matches')
insert_into_db(players_final, f'${club}-players')

'Data inserted successfully'

In [17]:
player_match_details = []

for index, row in players_final.iterrows():

    if row['MatchLogs']:
        url = 'https://fbref.com/'+ row['MatchLogs']
        fbref_playerid = row['fbref_playerid']
        season = row['Season']

        if url : 
            print(url)
            pmdf = get_player_data(url, season)
            if not pmdf is None: 
                pmdf['fbref_playerid'] = fbref_playerid
                player_match_details.append(pmdf)

player_details = pd.concat(player_match_details, ignore_index=True)
player_details = player_details.astype(str)
insert_into_db(player_details, f'${club}-player-details') 

https://fbref.com//en/players/6928979a/matchlogs/2022-2023/summary/Nicolo-Barella-Match-Logs
https://fbref.com//en/players/cd0fa27b/matchlogs/2022-2023/summary/Hakan-Calhanoglu-Match-Logs
https://fbref.com//en/players/f7036e1c/matchlogs/2022-2023/summary/Lautaro-Martinez-Match-Logs
https://fbref.com//en/players/75b86fb3/matchlogs/2022-2023/summary/Alessandro-Bastoni-Match-Logs
https://fbref.com//en/players/970b8574/matchlogs/2022-2023/summary/Federico-Dimarco-Match-Logs
https://fbref.com//en/players/b96b595c/matchlogs/2022-2023/summary/Francesco-Acerbi-Match-Logs
https://fbref.com//en/players/481c9ece/matchlogs/2022-2023/summary/Denzel-Dumfries-Match-Logs
https://fbref.com//en/players/e9c0c1b2/matchlogs/2022-2023/summary/Andre-Onana-Match-Logs
https://fbref.com//en/players/96e0490d/matchlogs/2022-2023/summary/Matteo-Darmian-Match-Logs
https://fbref.com//en/players/dd0daf32/matchlogs/2022-2023/summary/Henrikh-Mkhitaryan-Match-Logs
https://fbref.com//en/players/d0d8dd44/matchlogs/2022-20

OperationalError: (pymysql.err.OperationalError) (1241, 'Operand should contain 1 column(s)')
[SQL: INSERT INTO `$Internazionale-player-details` (`Date`, `Day`, `Comp`, `Round`, `Venue`, `Result`, `Squad`, `Opponent`, `Start`, `Pos`, `Min`, `Gls`, `Ast`, `PK`, `CrdY`, `CrdR`, `Match Report`, fbref_matchid, `Season`, fbref_playerid) VALUES (%(Date)s, %(Day)s, %(Comp)s, %(Round)s, %(Venue)s, %(Result)s, %(Squad)s, %(Opponent)s, %(Start)s, %(Pos)s, %(Min)s, %(Gls)s, %(Ast)s, %(PK)s, %(CrdY)s, %(CrdR)s, %(Match_Report)s, %(fbref_matchid)s, %(Season)s, %(fbref_playerid)s)]
[parameters: [{'Date': '2022-06-04', 'Day': 'Sat', 'Comp': 'UEFA Nations League', 'Round': 'League A', 'Venue': 'Home', 'Result': 'D 1–1', 'Squad': 'Italy', 'Opponent': 'Germany', 'Start': 'N', 'Pos': 'On matchday squad, but did not play', 'Min': '/en/matches/e1bf62bc/Italy-Germany-June-4-2022-UEFA-Nations-League', 'Gls': None, 'Ast': None, 'PK': None, 'CrdY': None, 'CrdR': None, 'Match_Report': None, 'fbref_matchid': Non